In [ ]:
import pandas as pd
from tqdm import tqdm
from yaml import safe_load
from joblib import Parallel, delayed
import warnings, json, ast ,os
import numpy as np

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)

### Convert JSON To CSV

In [212]:
# folder_path = '../odis_json'
# json_files = [i for i in os.listdir(folder_path) if i.endswith('.json')]

In [213]:
# cric_df_list = []
# counter = 1
# for file in tqdm(json_files):
#     with open(os.path.join(folder_path, file), 'r') as f:
#         df = pd.json_normalize(safe_load(f))
#         df['match_id'] = counter
#         cric_df_list.append(df)
#         counter += 1

# # Concatenate all dataframes in the list
# cric_df = pd.concat(cric_df_list, ignore_index=True)


In [214]:
# if not os.path.exists('../output'):
#     os.makedirs('../output')
# cric_df.to_csv('../output/json_extracted_data.csv',index=False)

### Extract Important columns

In [215]:
# df = pd.read_csv('../output/json_extracted_data.csv')

In [216]:
# df = df[df['info.gender'] == 'male'][['match_id','innings','info.outcome.winner','info.teams','info.venue']]

In [217]:
# # Convert "innings" column to a list of json
# df['innings'] = Parallel(n_jobs=-1)(delayed(ast.literal_eval)(x) for x in df['innings'])

In [218]:
# # Explode the 'innings' column
# df = df.explode('innings')

# # Extract 'team' separately
# df['innings_team'] = df['innings'].apply(lambda x: x.get('team') if isinstance(x, dict) else None)

# # Normalize the rest of the data
# df = pd.json_normalize(
#     df.to_dict(orient='records'),
#     record_path=['innings', 'overs', 'deliveries'],
#     meta=['match_id', 'innings_team', 'info.outcome.winner',
#           ['innings', 'overs', 'over'], 'info.teams', 'info.venue'],
#     errors='ignore'
# )

In [219]:
# # restructure and rename columns 

# df = df[['match_id','info.teams','innings_team','innings.overs.over','runs.total','wickets','info.outcome.winner',
#                                'info.venue']].rename(columns={'runs.total':'runs','info.outcome.winner':'winning_team','info.venue':'venue',
#                                                               'info.teams':'teams','innings_team':'batting_team','innings.overs.over':'over_number'}
# )

In [220]:
# df['teams'] = Parallel(n_jobs=-1)(delayed(ast.literal_eval)(x) for x in df['teams'])
# df['bowling_team'] = df.apply(
#     lambda row: [team for team in row['teams'] if team != row['batting_team']][0], axis=1
# )

In [221]:
# if not os.path.exists('../output'):
#     os.makedirs('../output')
# df.to_csv('../output/deliveries.csv',index=False)

### Data Preprocessing

In [222]:
df = pd.read_csv('../output/deliveries.csv')

In [223]:
df.head()

,match_id,teams,batting_team,over_number,runs,wickets,winning_team,venue,bowling_team
0,1,"['Australia', 'Pakistan']",Australia,0,0,NaN,Australia,"Brisbane Cricket Ground, Woolloongabba",Pakistan
1,1,"['Australia', 'Pakistan']",Australia,0,0,NaN,Australia,"Brisbane Cricket Ground, Woolloongabba",Pakistan
2,1,"['Australia', 'Pakistan']",Australia,0,0,NaN,Australia,"Brisbane Cricket Ground, Woolloongabba",Pakistan
3,1,"['Australia', 'Pakistan']",Australia,0,0,NaN,Australia,"Brisbane Cricket Ground, Woolloongabba",Pakistan
4,1,"['Australia', 'Pakistan']",Australia,0,1,NaN,Australia,"Brisbane Cricket Ground, Woolloongabba",Pakistan


#### Venues with atleast 10 matches

In [224]:
# Only select venues which had atleast 10 matches
venues = df['venue'].value_counts()[df['venue'].value_counts() > 3000].index.tolist()  # 300 x 10 = 3000

df = df[df['venue'].isin(venues)]

#### Top 10 teams

In [225]:
# top 10 teams
teams = ['Australia','India','Bangladesh','New Zealand','South Africa','England','West Indies','Afghanistan','Pakistan','Sri Lanka']

# choose from only top teams
df = df[df['batting_team'].isin(teams)]
df = df[df['bowling_team'].isin(teams)]

#### Get only first innings records

In [226]:
# For each match_id get only the first innings
df['match_team'] = df.groupby('match_id')['batting_team'].transform(  # row number() over(partition by match_id,batting_team)
    lambda x: pd.factorize(x)[0] + 1
)

In [227]:
df = df[df['match_team'] == 1]

#### Ball number column

In [228]:
# Create balls_number column by row number over match_id, batting_team and over_number
df['ball_number'] = df.groupby(['match_id', 'batting_team', 'over_number']).cumcount() + 1

#### Wickets, players dismissed, wickets left column

In [229]:
# make wicket as 0,1 column
df.insert(7,'wicket',np.where(df['wickets'].notna(),1,0))
df.drop(columns=['wickets','teams'],axis=1,inplace=True)

In [230]:
# players/batsmen dismissed & wickets left column
df['players_dismissed'] = df.groupby(['match_id','batting_team']).cumsum()['wicket'] # cumulative sum of player_dismissed by match_id
df['wickets_left'] = 10 - df['players_dismissed'] # new column of wickets left
df.drop(columns='wicket',axis=1,inplace=True)

#### Current score column

In [231]:
# current score
df['current_score'] = df.groupby('match_id').cumsum()['runs'] # find cumsum of runs for each ball

#### Balls bowled, balls left columns

In [232]:
# Balls bowled
df['balls_bowled'] = (df['over_number'].astype('int')*6) + df['ball_number'].astype('int') # overs * 6 + balls

In [233]:
# Balls left
df['balls_left'] = 300 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x) # remove -ve values if any

#### Current run rate column

In [234]:
# Current run rate column
df['crr'] = (df['current_score']*6)/df['balls_bowled'] # current_run_rate = current_score*6/balls_bowled

#### Runs scored in last 5 overs column

In [235]:
# Runs scored in last 5 overs
match_ids = df['match_id'].unique() # get array of unique match ids

groups = df.groupby('match_id') # create groups for each match

last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist()) # roll through last 30 balls
df['last_five'] = last_five

#### Match wise total runs

In [236]:
# total runs of first innings (class-label)
df.groupby('match_id').sum()['runs'].reset_index()

df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

#### Finalize Dataframe

In [237]:
df = df[['batting_team', 'bowling_team', 'venue', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five', 'runs_x']]
df.dropna(inplace=True)

In [238]:
df.to_csv('../output/deliveries_final.csv',index=False)

In [239]:
df.tail()

,batting_team,bowling_team,venue,current_score,balls_left,wickets_left,crr,last_five,runs_x
387217,Sri Lanka,Australia,Pallekele International Cricket Stadium,194,61,1,4.870293,24.0,195
387218,Sri Lanka,Australia,Pallekele International Cricket Stadium,194,60,1,4.850000,23.0,195
387219,Sri Lanka,Australia,Pallekele International Cricket Stadium,195,59,1,4.854772,24.0,195
387220,Sri Lanka,Australia,Pallekele International Cricket Stadium,195,59,1,4.854772,23.0,195
387221,Sri Lanka,Australia,Pallekele International Cricket Stadium,195,58,0,4.834711,22.0,195
